In [5]:
import pandas as pd 
import numpy as np
from pandas_datareader import data as pdr
import yfinance as yf #yahoo finance stock data
import datetime
import matplotlib.pyplot as plt
from math import sqrt

import tensorflow
from tensorflow import keras

from sklearn.linear_model import LinearRegression,ElasticNet
from sklearn.ensemble import RandomForestRegressor

stock="GOOG"
period=14

In [48]:
end=datetime.date(2020,1,1)

base=255*3
delta_d=datetime.timedelta(days=base)
delta_m=datetime.timedelta(minutes=base)
delta_t=datetime.timedelta(minutes=(base*30))
delta_h=datetime.timedelta(hours=base)
delta_w=datetime.timedelta(weeks=base)

In [50]:
(end-delta_m),end

(datetime.date(2020, 1, 1), datetime.date(2020, 1, 1))

In [42]:
min=yf.download(tickers=stock,startdate=end-delta_m,enddate=end,interval="1m") #period="3d",
thirt_min=yf.download(tickers=stock,startdate=end-delta_t,enddate=end, interval="30m")#,period="31d"
hour=yf.download(tickers=stock,startdate=end-delta_h,enddate=end,interval="1h")#,period="122d"
daily=yf.download(tickers=stock,startdate=end-delta_d,enddate=end,interval="1d")#,period="765d"
week=yf.download(tickers=stock,startdate=end-delta_w,enddate=end,interval="1wk")#,period="3650d"

[*********************100%***********************]  1 of 1 completed

1 Failed download:
- GOOG: 1m data not available for startTime=1092922200 and endTime=1604259343. Only 7 days worth of 1m granularity data are allowed to be fetched per request.
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- GOOG: 15m data not available for startTime=1092922200 and endTime=1604259343. The requested range must be within the last 60 days.
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- GOOG: 1h data not available for startTime=1092922200 and endTime=1604259344. The requested range must be within the last 730 days.
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [41]:
min.shape,thirt_min.shape,daily.shape

((1164, 6), (403, 6), (765, 6))

In [7]:
def Prepare_dataset(df):
    from sklearn.model_selection import train_test_split

    typ_price=(df.Close + df.High+df.Low)/3
    mf=typ_price*df.Volume

    pos_flow=[]
    neg_flow=[]

    for i in range(0,len(typ_price)):
        if typ_price[i]>typ_price[i-1]:
            pos_flow.append(mf[i-1])
            neg_flow.append(0)
        elif typ_price[i]<typ_price[i-1]:
            pos_flow.append(0)
            neg_flow.append(mf[i-1])
        else:
            pos_flow.append(0)
            neg_flow.append(0)
        
    pos_mf=[]
    neg_mf=[]

    for i in range(period-1,len(pos_flow)):
        pos_mf.append(sum(pos_flow[i+1-period:i]))

    for i in range(period-1,len(neg_flow)):
        neg_mf.append(sum(neg_flow[i+1-period:i]))

    mf_ratio=np.array(pos_mf)/np.array(neg_mf)

    mfi=100-100/(1+mf_ratio)
    df=df.iloc[13:,:]

    df["MFI"]=mfi

    y=df.iloc[:-1,:]["Adj Close"].shift(-1)
    y.fillna(method='ffill', inplace=True)
    X=df.iloc[:-1,:].fillna(method='ffill')

    return train_test_split(X, y, test_size=0.4, random_state=0)

def Rmse(y,y_pred):
    return np.sqrt(((y_pred - y) ** 2).mean())

In [10]:
datasets={}
datasets["1m"]=Prepare_dataset(min)
datasets["30m"]=Prepare_dataset(thirt_min)
datasets["1h"]=Prepare_dataset(hour)
datasets["D"]=Prepare_dataset(daily)
datasets["W"]=Prepare_dataset(week)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:30: RuntimeWarning: divide by zero encountered in true_divide


In [11]:
from sklearn.linear_model import LinearRegression

In [12]:
result=pd.DataFrame(columns=["1m","30m","1h","D","W"])

for key in datasets:
    X_train=datasets[key][0]
    X_test=datasets[key][1]
    y_train=datasets[key][2]
    y_test=datasets[key][3]

    elastic=ElasticNet().fit(X_train, y_train)
    elastic_preds=elastic.predict(X_test)

    result.loc["Elastic net",key]=[Rmse(y_test,elastic_preds)]

    rf=RandomForestRegressor(n_estimators=100,max_depth=12,random_state=0).fit(X_train, y_train)
    rf_preds=rf.predict(X_test)

    result.loc["Random forest",key]=[Rmse(y_test,rf_preds)]


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 310.07808451129495, tolerance: 126.0940649360505
  positive)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 132.81549874525444, tolerance: 95.94946076772237
  positive)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 18484.700906057435, tolerance: 288.375622044715
  positive)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 77423.5920

In [13]:
result

,1m,30m,1h,D,W
Elastic net,[5.120252196919519],[9.882863502639127],[9.211757395650759],[23.45388415427078],[26.556752412361632]
Random forest,[5.325712002910218],[8.964527937752607],[11.022239225635754],[26.193195325498692],[28.673122278499992]
